# Movie Recommendation System : notes

--> Upload "u.data" and "u.item"
"sep='\t'" ---> the dataset: data in the file is separated by using a special charater which is "tab space".
df --> DataFrame
--> Naming the columns with names
nunique --> finds unique value with respect to some id






In [50]:
import numpy as np
import pandas as pd
import warnings

In [51]:
# removing the warnings that would unnecessary come
warnings.filterwarnings('ignore')

In [52]:
#creating a "dataframe" - "df"
#renaming the columns using the list "columns_name"
# sep="\t" ---> the dataset: data in the file is separated by using a special charater which is "tab space"
# upload "u.data" and "u.item" here, then proceed.

# working with the FIRST DATASET : u.data

columns_name=['user_id','item_id','rating','timestamp']
df=pd.read_csv('u.data',sep="\t",names=columns_name)

In [53]:
# using the function head() - we can show the top 5 entries.

df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [54]:
# "shape" of dataset (u.data - df) - 100000 rows and 4 columns

df.shape

(100000, 4)

In [55]:
# how many users are there

df['user_id']

0        196
1        186
2         22
3        244
4        166
        ... 
99995    880
99996    716
99997    276
99998     13
99999     12
Name: user_id, Length: 100000, dtype: int64

In [56]:
# how many unique users are there
#  nunique --> finds unique value with respect to some id

df['user_id'].nunique()

943

So there are 943 unique users who have rated the movies.

In [57]:
# how many unique movies have been rated by users

df['item_id'].nunique()

1682

NOTES:
So 943 unique users have rated 1682 unique number of movies.

In [58]:
# Working with the SECOND DATASET : u.item

# Naming it as "movies_title"

movies_title=pd.read_csv('u.item',sep="\|",header=None)

In [59]:
# shape of dataset (u.item - movies_title)
# It has 1682 rows and 24 columns

movies_title.shape


(1682, 24)

In [60]:
# Renaming the columns and taking only 2 fields : 0th col & 1st col
# movies_title --> movies_titles (RENAMING)
# 0th col --> item_id (RENAMING)
# 1st col --> title (RENAMING)

movies_titles=movies_title[[0,1]]
movies_titles.columns=["item_id","title"]
movies_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


Now, both the datasets have a same column called "item_id". 

Refer Out[11] and Out[23].

Hence we are going to merge the 2 datasets.

In [61]:
# Merging 2 datasets : using merge func in pandas (so, "pd")

# merge(dataset1, dataset2, on = "common column")

df=pd.merge(df,movies_titles,on="item_id")

In [62]:
# displaying the merged dataset

df

,user_id,item_id,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)
...,...,...,...,...,...
99995,840,1674,4,891211682,Mamma Roma (1962)
99996,655,1640,3,888474646,"Eighth Day, The (1996)"
99997,655,1637,3,888984255,Girls Town (1996)
99998,655,1630,3,887428735,"Silence of the Palace, The (Saimt el Qusur) (1..."


The 2 datasets have been merged into 1 with item_id as common column.

From this we understand that user 196 [user_id] has given the movie 242 [item_id] of the title "Kolya(1996)" [title]  with a rating of 3 [rating] at a timestamp.

In [63]:
# displaying the last 5 entries

df.tail()

,user_id,item_id,rating,timestamp,title
99995,840,1674,4,891211682,Mamma Roma (1962)
99996,655,1640,3,888474646,"Eighth Day, The (1996)"
99997,655,1637,3,888984255,Girls Town (1996)
99998,655,1630,3,887428735,"Silence of the Palace, The (Saimt el Qusur) (1..."
99999,655,1641,3,887427810,Dadetown (1995)


In [64]:
# Calculating "average ratings" as "ratings" : grouped by movie name and average is calculated by using mean() func 
# for "rating" column.
# This is done for every movie.

ratings=pd.DataFrame(df.groupby('title').mean()['rating'])

In [65]:
# displaying the 1st five entries

ratings.head()

,rating
title,
'Til There Was You (1997),2.333333
1-900 (1994),2.600000
101 Dalmatians (1996),2.908257
12 Angry Men (1957),4.344000
187 (1997),3.024390


Note : The rating displayed here is average rating.

In [66]:
# how many people have rated which movie

ratings['num of ratings']=pd.DataFrame(df.groupby('title').count()['rating'])

Create the recommendar System :

In [67]:
df.head()

,user_id,item_id,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


Our current dataset has all these entries.

In [68]:
# Creating a dataset "MOVIEMAT" with new format
# Columns : User ID, Movie 1, Movie 2, Movie 3, etc...
# Rows:  User IDs for  User ID col and RATINGS given by that user for that respective movie as table entries. --> values:"rating"
# If a user has not given a rating for that movie, the entry becomes "NaN"

moviemat=df.pivot_table(index="user_id",columns="title",values="rating")

In [69]:
# disp top 5 entries
moviemat.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN


In [70]:

# Just taking "starwars" movie into consideration, we are building a customised recommender system.

starwars_user_ratings=moviemat['Star Wars (1977)']

In [71]:

# disp top 20 entries

starwars_user_ratings.head(20)

user_id
1     5.0
2     5.0
3     NaN
4     5.0
5     4.0
6     4.0
7     5.0
8     5.0
9     5.0
10    5.0
11    NaN
12    4.0
13    5.0
14    5.0
15    5.0
16    NaN
17    NaN
18    4.0
19    NaN
20    3.0
Name: Star Wars (1977), dtype: float64

In [72]:
# Correlation : correlation values of other movies that are similar to starwars
# Saving these values in "similar_to_starwars"
# Using function .corrwith() in pandas

similar_to_starwars=moviemat.corrwith(starwars_user_ratings)

In [73]:

# displaying everything

similar_to_starwars

title
'Til There Was You (1997)                0.872872
1-900 (1994)                            -0.645497
101 Dalmatians (1996)                    0.211132
12 Angry Men (1957)                      0.184289
187 (1997)                               0.027398
                                           ...   
Young Guns II (1990)                     0.228615
Young Poisoner's Handbook, The (1995)   -0.007374
Zeus and Roxanne (1997)                  0.818182
unknown                                  0.723123
Á köldum klaka (Cold Fever) (1994)            NaN
Length: 1664, dtype: float64


We can see that, for all the movies, the correlation values are calculated. There are positive values and negative values denoting that there is positive correlation and negative correlation respectively with respect to starwars movie's rating.

There is also "NaN" values filling the place because there is no rating entry found for the movie.

In [74]:
# Purpose : to remove "NaN" values from the correlation func as they will hinder our recommendation process 
# So we are creating another NEW correlation func = "corr_starwars"

corr_starwars=pd.DataFrame(similar_to_starwars,columns=['correlation'])


In [75]:
# func to drop NaN : .dropna(inplace=True)

corr_starwars.dropna(inplace=True)

In [76]:
# Display "corr_starwars" to check if NaN values are removed
corr_starwars

,correlation
title,
'Til There Was You (1997),0.872872
1-900 (1994),-0.645497
101 Dalmatians (1996),0.211132
12 Angry Men (1957),0.184289
187 (1997),0.027398
...,...
Young Guns (1988),0.186377
Young Guns II (1990),0.228615
"Young Poisoner's Handbook, The (1995)",-0.007374


From the above table we see that, "NaN" values are removed.

In [77]:

# disp top 5 entries
corr_starwars.head()

,correlation
title,
'Til There Was You (1997),0.872872
1-900 (1994),-0.645497
101 Dalmatians (1996),0.211132
12 Angry Men (1957),0.184289
187 (1997),0.027398


In [78]:

# We now want the closest positive correlation ( perfect correlation = 1) to starwars movie

corr_starwars.sort_values('correlation',ascending=False).head(10)


,correlation
title,
Hollow Reed (1996),1.0
Commandments (1997),1.0
Cosi (1996),1.0
No Escape (1994),1.0
Stripes (1981),1.0
Star Wars (1977),1.0
Man of the Year (1995),1.0
"Beans of Egypt, Maine, The (1994)",1.0
"Old Lady Who Walked in the Sea, The (Vieille qui marchait dans la mer, La) (1991)",1.0



There is a problem in this perfect correlation value that we are getting. This "1.0" may have come because only 5 people have given a full rating.

Therefore :
          We need to set a threshold / condition that we would only consider the movies' rating rated by atleast (min) 100 people inorder for our recommendation system to be accurate.

Because, we cannot consider a rating rated by only 5-10 people.

In [79]:
# Disp "ratings" inorder to set the condition
ratings

,rating,num of ratings
title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41
...,...,...
Young Guns II (1990),2.772727,44
"Young Poisoner's Handbook, The (1995)",3.341463,41
Zeus and Roxanne (1997),2.166667,6



Here, "num of ratings" is the number of people who have rated the movie.

eg : 5 people have rated "1-900(1994)" movie.

And "rating" --> average rating

In [80]:

# Joining the column "num of ratings" to "corr_starwars"
corr_starwars=corr_starwars.join(ratings['num of ratings'])


In [81]:
#disp joined table
corr_starwars

,correlation,num of ratings
title,,
'Til There Was You (1997),0.872872,9
1-900 (1994),-0.645497,5
101 Dalmatians (1996),0.211132,109
12 Angry Men (1957),0.184289,125
187 (1997),0.027398,41
...,...,...
Young Guns (1988),0.186377,101
Young Guns II (1990),0.228615,44
"Young Poisoner's Handbook, The (1995)",-0.007374,41


In [82]:

# disp tp 5 val

corr_starwars.head()

,correlation,num of ratings
title,,
'Til There Was You (1997),0.872872,9
1-900 (1994),-0.645497,5
101 Dalmatians (1996),0.211132,109
12 Angry Men (1957),0.184289,125
187 (1997),0.027398,41


In [83]:

# (num of ratings > 100) and (.sort_values(corr, descending order))

corr_starwars[corr_starwars['num of ratings']>100].sort_values('correlation',ascending=False)


,correlation,num of ratings
title,,
Star Wars (1977),1.000000,583
"Empire Strikes Back, The (1980)",0.747981,367
Return of the Jedi (1983),0.672556,507
Raiders of the Lost Ark (1981),0.536117,420
Austin Powers: International Man of Mystery (1997),0.377433,130
...,...,...
"Edge, The (1997)",-0.127167,113
As Good As It Gets (1997),-0.130466,112
Crash (1996),-0.148507,128



The output here is the PREDICTIONS for the movie "starwars" from nearest to farthest.

Now, this is the PERFECT RECOMMENDATION SYSTEM for starwars.

The next step is to write a GENERALIZED FUNCTION to recommend movies for any input movie.


In [84]:

# Generalized func for the movie recommendation system:

def predict_movies(movie_name):
    movie_user_ratings=moviemat[movie_name]
    similar_to_movie=moviemat.corrwith(movie_user_ratings)
    corr_movie=pd.DataFrame(similar_to_movie,columns=['correlation'])
    corr_movie.dropna(inplace=True)
    corr_movie=corr_movie.join(ratings['num of ratings'])
    
    predictions=corr_movie[corr_movie['num of ratings']>100].sort_values('correlation',ascending=False)
    
    return predictions

In [85]:

# Giving input to "predict_movies()" func with the movie name Titanic and storing it in "predict_my_movie"

predict_my_movie=predict_movies("Titanic (1997)")

In [86]:

# Disp top 5 movies that are closest to (recommended) Titanic(1997)

predict_my_movie.head()

,correlation,num of ratings
title,,
Titanic (1997),1.000000,350
"River Wild, The (1994)",0.497600,146
"Abyss, The (1989)",0.472103,151
Bram Stoker's Dracula (1992),0.443560,120
True Lies (1994),0.435104,208


In [87]:

# My Program

print(" Enter a movie name :")
my_movie = input()
predict_my_movie=predict_movies(my_movie)
print("The movies that are recommended to watch are :")
predict_my_movie.head()


 Enter a movie name :
Titanic (1997)
The movies that are recommended to watch are :


,correlation,num of ratings
title,,
Titanic (1997),1.000000,350
"River Wild, The (1994)",0.497600,146
"Abyss, The (1989)",0.472103,151
Bram Stoker's Dracula (1992),0.443560,120
True Lies (1994),0.435104,208
